<a href="https://colab.research.google.com/github/yeonjoo7/final_project/blob/main/Custom_dataset_format(AIHub_sidewalk_datataset).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. DATASET 구성
사용하려는 라이브러리에 따라 맞는 dataset format을 변환하기 위해 정리한 내용

In [ ]:
import torch, torchvision
import numpy as np
import pandas as pd

import cv2
import json
import itertools
import os
from glob import glob
import xml.etree.ElementTree as ET

print(torch.__version__)

1.7.0+cu101


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


## xml파일에서 label, Bounding Box 좌표 추출하는 함수

In [ ]:
# 정은님 함수

path = '/content/drive/MyDrive/sidewalk_bbox/Bbox_1/Bbox_0001'

def get_box_df(filename, xmlname):
  xml_data = open((os.path.join(path,xmlname)), 'r').read()
  root = ET.XML(xml_data)
  df = pd.DataFrame(columns=('label','occluded', 'xtl', 'ytl', 'xbr', 'ybr', 'z_order'))
  for image in root.getchildren():
    if image.attrib.get('name') == filename :
      print(filename)
      dic = {'label':[],'occluded':[], 'xtl':[], 'ytl':[], 'xbr':[], 'ybr':[], 'z_order':[]}
      for i in range(0,len(image.getchildren())):
          obj = image.getchildren()[i]
          for col in ['label','occluded', 'xtl', 'ytl', 'xbr', 'ybr', 'z_order']:
            dic[col].append(obj.attrib[col])
      for col in ['label','occluded', 'xtl', 'ytl', 'xbr', 'ybr', 'z_order']:
        df[col] = dic[col]
  return df
data = get_box_df('MP_SEL_000001.jpg','0617_01.xml')
data

MP_SEL_000001.jpg


,label,occluded,xtl,ytl,xbr,ybr,z_order
0,tree_trunk,0,1272.39,451.90,1331.40,670.79,0
1,movable_signage,1,1082.78,356.68,1147.89,491.46,0
2,tree_trunk,0,1157.03,277.90,1210.30,565.71,0
3,tree_trunk,0,1094.21,331.56,1131.90,493.75,0
4,car,0,1672.90,382.30,1728.00,416.00,0
5,traffic_sign,0,1851.90,346.60,1886.20,380.31,0
6,car,1,1768.60,423.00,1914.00,499.00,0
7,car,1,1634.90,499.70,1920.00,857.50,0
8,car,1,1692.20,411.90,1814.00,478.00,0
9,movable_signage,1,1206.14,303.00,1289.52,542.86,0


## xml 바로 json으로 변환
(단순 변환이기때문에 AI hub에서 받은 dataset xml로는 COCO json변환이 안됨)

In [ ]:
!pip install xmltodict

In [ ]:
import xmltodict
import json

xmlname = 'Bbox_0131/0702_08.xml'
xml_data = open((os.path.join(path3,xmlname)), 'r').read()

# xmltodict.parse() 함수는 OrderedDict 으로 return

result = xmltodict.parse(xml_data)

dictionary = json.loads(json.dumps(result))

dictionary
#  아래와 같은 형식

# {'annotations': {'image': [{'@height': '1080',
#     '@id': '0',
#     '@name': 'MP_SEL_018001.jpg',
#     '@width': '1920',
#     'box': [{'@label': 'car',
#       '@occluded': '1',
#       '@xbr': '1920.00',
#       '@xtl': '1573.20',
#       '@ybr': '615.00',
#       '@ytl': '373.00',
#       '@z_order': '0'},
#      {'@label': 'person',
#       '@occluded': '1',
#       '@xbr': '701.30',
#       '@xtl': '575.63',
#       '@ybr': '641.10',
#       '@ytl': '458.73',
#       '@z_order': '1'}, ...
#       'meta': {'dumped': '2019-11-29 04:03:25.280548+03:00',
#    'task': {'bugtracker': None,
#     'created': '2019-11-22 06:36:26.055375+03:00',
#     'flipped': 'False',
#     'frame_filter': '0',
#     'id': '5070',
#     'labels': {'label': [{'attributes': None, 'name': 'wheelchair'},
#       {'attributes': None, 'name': 'truck'},
#       {'attributes': None, 'name': 'tree_trunk'},
#       {'attributes': None, 'name': 'traffic_sign'},
#       {'attributes': None, 'name': 'traffic_light'},
#       {'attributes': None, 'name': 'table'},
#       {'attributes': None, 'name': 'stroller'},
#       {'attributes': None, 'name': 'stop'},
#       {'attributes': None, 'name': 'scooter'},
#       {'attributes': None, 'name': 'potted_plant'},
#       {'attributes': None, 'name': 'pole'},
#       {'attributes': None, 'name': 'person'},
#       {'attributes': None, 'name': 'parking_meter'},
#       {'attributes': None, 'name': 'movable_signage'},
#       {'attributes': None, 'name': 'motorcycle'},
#       {'attributes': None, 'name': 'kiosk'},
#       {'attributes': None, 'name': 'fire_hydrant'},
#       {'attributes': None, 'name': 'dog'},
#       {'attributes': None, 'name': 'chair'},
#       {'attributes': None, 'name': 'cat'},
#       {'attributes': None, 'name': 'carrier'},
#       {'attributes': None, 'name': 'car'},
#       {'attributes': None, 'name': 'bus'},
#       {'attributes': None, 'name': 'bollard'},
#       {'attributes': None, 'name': 'bicycle'},
#       {'attributes': None, 'name': 'bench'},
#       {'attributes': None, 'name': 'barricade'}]},
#     'mode': 'annotation',
#     'name': '0702_08(최종검수)',
#     'overlap': '0',
#     'owner': {'email': 'aiadmin@testworks.co.kr', 'username': 'aiadmin'},
#     'segments': {'segment': {'id': '4868',
#       'start': '0',
#       'stop': '285',
#       'url': 'http://112.169.146.227:8080/?id=4868'}},
#     'size': '286',
#     'start_frame': '0',
#     'stop_frame': '0',
#     'updated': '2019-11-22 11:29:05.790985+03:00'}},
#   'version': '1.1'}}

## xml 파일을 VOC xml 형식으로 변환

In [ ]:
# 참고한 코드 출처(voc xml 생성): https://ballentain.tistory.com/8

from xml.etree.ElementTree import Element, SubElement, ElementTree

img_path = '/content/drive/MyDrive/Bbox_0002'
xml_path = '/content/drive/MyDrive/Bbox_0002/original_xml'

def new_voc_xml(filename, xmlname):

  # xml data로 df 만들기
  xml_data = open((os.path.join(xml_path,xmlname)), 'r').read()
  root = ET.XML(xml_data)
  df = pd.DataFrame(columns=('label','occluded', 'xtl', 'ytl', 'xbr', 'ybr', 'z_order'))
  for image in root.getchildren():
    if image.attrib.get('name') == filename :
      print(filename)
      dic = {'label':[],'occluded':[], 'xtl':[], 'ytl':[], 'xbr':[], 'ybr':[], 'z_order':[]}
      for i in range(0,len(image.getchildren())):
          obj = image.getchildren()[i]
          for col in ['label','occluded', 'xtl', 'ytl', 'xbr', 'ybr', 'z_order']:
            dic[col].append(obj.attrib[col])
      for col in ['label','occluded', 'xtl', 'ytl', 'xbr', 'ybr', 'z_order']:
        df[col] = dic[col]

  # 위의 df를 활용하여 새로운 VOC xml format 문서 만들기 (xml은 전부 str타입으로 맞춰줘야함)
  width = 1920
  height = 1080
  root = Element('annotation')
  SubElement(root, 'folder').text = 'ai_hub_images'
  SubElement(root, 'filename').text = filename
  source = SubElement(root, 'source')
  SubElement(source, 'database').text = 'Unknown'   # Pascal VOC dataset내 위치

  size = SubElement(root, 'size')
  SubElement(size, 'width').text = str(width)
  SubElement(size, 'height').text = str(height)
  SubElement(size, 'depth').text = '3'            # 이미지의 채널 수

  SubElement(root, 'segmented').text = '0'      # Class or Object segmentation을 위한 정보

  for row in df.itertuples():
    print(row)
    label = row[1]
    print(label)
    obj = SubElement(root, 'object')
    SubElement(obj, 'name').text = str(label)
    SubElement(obj, 'pose').text = 'Unspecified'  # 이미지 내 객체의 위치
    SubElement(obj, 'truncated').text = str(row[2]) # 이미지 내 객체의 잘림 여부
    SubElement(obj, 'difficult').text = '0'       # 이미지 내 객체의 식별 가능 여부
    bbox = SubElement(obj, 'bndbox')
    print(row[3], row[6])
    SubElement(bbox, 'xmin').text = str(row[3])
    SubElement(bbox, 'ymin').text = str(row[4])
    SubElement(bbox, 'xmax').text = str(row[5])
    SubElement(bbox, 'ymax').text = str(row[6])                                                      

  tree = ElementTree(root)
  print(tree)
  tree.write('/content/drive/MyDrive/Bbox_0002/'+filename[:-4]+'.xml')

# 소폴더 하나로 테스트
temp = []
for file_names in os.walk(img_path):
  temp.append(file_names)
file_names = temp[0][2]
img_files = []
for file_name in file_names:
    if file_name.endswith('.jpg'):
      img_files.append(file_name)

# 구글 코랩에 저장하는 코드
# with open(path.join(notebooks_base_dir, "file.txt"), "w") as f:
#     f.write("contents")

In [ ]:
print(img_files)

for img in img_files:
  new_voc_xml(img,'0617_02.xml')

In [ ]:
cnt_img = []
cnt_xml = []
for dir_path, dir_names, file_names in os.walk(path):
  for file_name in file_names:
    if file_name.endswith('.jpg'):
      cnt_img.append(file_name)
    elif file_name.endswith('.xml'):
      cnt_xml.append(file_name)

print(len(cnt_img))
print(len(cnt_xml))
if '0617_02.xml' in cnt_xml:
  print('Success!')

195
196
Success!


## 기존 xml파일로 새로운 YOLOv3 Darknet txt 생성

In [ ]:
# 출처: http://daddynkidsmakers.blogspot.com/2020/05/yolo.html

import os
import glob
from os import listdir, getcwd
from tokenize import tokenize, untokenize, NUMBER, STRING, NAME, OP

# Modify the below parametercategoryCount = 2names = ['con_eq', 'worker']
width = 500height = 500
# Programroot = ''def createFolders(root):
    try:
        os.mkdir(root)
        os.mkdir(root + 'images')
        os.mkdir(root + 'labels')
    except:
        return    return
def saveNames(root, file):
    try:
        fileName = open(root + file, "w")
        for index in range(categoryCount):
            fileName.write(names[index] + '\n')
    except:
        return    return
def saveLabel(folder, filename, imageName, category, object, w, h, BBoxList):
    try:
        file = open(folder + filename, "w")

        for BBox in BBoxList:  # type: object            objWidth = float(BBox[2] - BBox[0]) / float(w)
            objHeight = float(BBox[3] - BBox[1]) / float(h)
            centerX = objWidth / 2 + float(BBox[0]) / float(w)
            centerY = objHeight / 2 + float(BBox[1]) / float(h)

            file.write(str(category) + ' ')
            file.write('{:.6f}'.format(centerX) + ' ')
            file.write('{:.6f}'.format(centerY) + ' ')
            file.write('{:.6f}'.format(objWidth) + ' ')
            file.write('{:.6f}'.format(objHeight) + '\n')
    except:
        return    return
def convertBBoxToDarknet(root, labelDir, category, w, h):
    labelList = glob.glob(os.path.join(labelDir, '*.*'))
    for filename in labelList:
        BBoxList = []
        try:
            file = open(filename, "r")
            count = file.readline()
            count = count.rstrip('\n')
            num = int(count)
            for line in range(num):
                BBox = file.readline()
                BB = BBox.split()
                points = []
                for pt in BB:
                    points.append(int(pt))
                BBoxList.append(points)
        except:
            continue
        object = names[category]

        name = os.path.basename(filename)
        name, extension = os.path.splitext(name)
        labelFile = name + '.txt'        saveLabel(root + 'labels/', labelFile, name, category, object, w, h, BBoxList)
    return
def saveTrainTestsets(root, train, test, ratio):
    try:
        fileTest = open(root + train, "w")
        fileTrain = open(root + test, "w")

        imageDir = os.path.join(root + 'images/')
        imageList = glob.glob(os.path.join(imageDir, '*.jpg'))

        labelDir = os.path.join(root + 'labels/')
        labelList = glob.glob(os.path.join(labelDir, '*.*'))

        no = 0        count = len(labelList)
        trainCount = int(float(count) * ratio)
        for file in imageList:

            name = os.path.basename(file)
            name, extension = os.path.splitext(name)
            labelFile = root + 'labels/' + name + '.txt'
            if os.path.isfile(labelFile) == False:
                continue
            if no < trainCount:
                fileTest.write(file + '\n')
            else:
                fileTrain.write(file + '\n')
            no = no + 1    except:
        return    return
def saveYOLOdatafile(root):
    try:
        file = open(root + 'detector.data', "w")

        file.write('classes=' + str(categoryCount) + '\n')
        file.write('train=custom_data/train.txt' + '\n')
        file.write('valid=custom_data/test.txt' + '\n')
        file.write('names=custom_data/custom.names' + '\n')
        file.write('backup=backup/' + '\n')
    except:
        return    return
wd = getcwd()
root = wd + '/custom_data/'createFolders(root)

saveNames(root, "custom.names")

for index in range(categoryCount):
    labelDir = getcwd() + '/'    labelDir = labelDir + os.path.join(r'BBox/Labels/', '%03d' % (index + 1))
    convertBBoxToDarknet(root, labelDir, index, width, height)

saveTrainTestsets(root, 'train.txt', 'test.txt', 0.8)

saveYOLOdatafile(root)

## VOC xml to COCO json

In [ ]:
# 출처: https://github.com/soumenpramanik/Convert-Pascal-VOC-to-COCO/blob/master/convertVOC2COCO.py

import os
import xml.etree.ElementTree as ET
import xmltodict
import json
from xml.dom import minidom
from collections import OrderedDict


#attrDict = {"images":[{"file_name":[],"height":[], "width":[],"id":[]}], "type":"instances", "annotations":[], "categories":[]}

#xmlfile = "000023.xml"


def generateVOC2Json(rootDir,xmlFiles):
	attrDict = dict()
	#images = dict()
	#images1 = list()
	attrDict["categories"]=[{"supercategory":"none","id":1,"name":"header"},
			        {"supercategory":"none","id":2,"name":"row"},
			        {"supercategory":"none","id":3,"name":"logo"},
			        {"supercategory":"none","id":4,"name":"item_name"},
				{"supercategory":"none","id":5,"name":"item_desc"},
				{"supercategory":"none","id":6,"name":"price"},
				{"supercategory":"none","id":7,"name":"total_price_text"},
				{"supercategory":"none","id":8,"name":"total_price"},
				{"supercategory":"none","id":9,"name":"footer"}
			      ]
	images = list()
	annotations = list()
	for root, dirs, files in os.walk(rootDir):
		image_id = 0
		for file in xmlFiles:
			image_id = image_id + 1
			if file in files:
				
				#image_id = image_id + 1
				annotation_path = os.path.abspath(os.path.join(root, file))
				
				#tree = ET.parse(annotation_path)#.getroot()
				image = dict()
				#keyList = list()
				doc = xmltodict.parse(open(annotation_path).read())
				#print doc['annotation']['filename']
				image['file_name'] = str(doc['annotation']['filename'])
				#keyList.append("file_name")
				image['height'] = int(doc['annotation']['size']['height'])
				#keyList.append("height")
				image['width'] = int(doc['annotation']['size']['width'])
				#keyList.append("width")

				#image['id'] = str(doc['annotation']['filename']).split('.jpg')[0]
				image['id'] = image_id
				print "File Name: {} and image_id {}".format(file, image_id)
				images.append(image)
				# keyList.append("id")
				# for k in keyList:
				# 	images1.append(images[k])
				# images2 = dict(zip(keyList, images1))
				# print images2
				#print images

				#attrDict["images"] = images

				#print attrDict
				#annotation = dict()
				id1 = 1
				if 'object' in doc['annotation']:
					for obj in doc['annotation']['object']:
						for value in attrDict["categories"]:
							annotation = dict()
							#if str(obj['name']) in value["name"]:
							if str(obj['name']) == value["name"]:
								#print str(obj['name'])
								#annotation["segmentation"] = []
								annotation["iscrowd"] = 0
								#annotation["image_id"] = str(doc['annotation']['filename']).split('.jpg')[0] #attrDict["images"]["id"]
								annotation["image_id"] = image_id
								x1 = int(obj["bndbox"]["xmin"])  - 1
								y1 = int(obj["bndbox"]["ymin"]) - 1
								x2 = int(obj["bndbox"]["xmax"]) - x1
								y2 = int(obj["bndbox"]["ymax"]) - y1
								annotation["bbox"] = [x1, y1, x2, y2]
								annotation["area"] = float(x2 * y2)
								annotation["category_id"] = value["id"]
								annotation["ignore"] = 0
								annotation["id"] = id1
								annotation["segmentation"] = [[x1,y1,x1,(y1 + y2), (x1 + x2), (y1 + y2), (x1 + x2), y1]]
								id1 +=1

								annotations.append(annotation)
				
				else:
					print "File: {} doesn't have any object".format(file)
				#image_id = image_id + 1
				
			else:
				print "File: {} not found".format(file)
			

	attrDict["images"] = images	
	attrDict["annotations"] = annotations
	attrDict["type"] = "instances"

	#print attrDict
	jsonString = json.dumps(attrDict)
	with open("receipts_valid.json", "w") as f:
		f.write(jsonString)

# rootDir = "/netscratch/pramanik/OBJECT_DETECTION/detectron/lib/datasets/data/Receipts/Annotations"
# for root, dirs, files in os.walk(rootDir):
# 	for file in files:
# 		if file.endswith(".xml"):
# 			annotation_path = str(os.path.abspath(os.path.join(root,file)))
# 			#print(annotation_path)
# 			generateVOC2Json(annotation_path)
trainFile = "/netscratch/pramanik/OBJECT_DETECTION/valid.txt"
trainXMLFiles = list()
with open(trainFile, "rb") as f:
	for line in f:
		fileName = line.strip()
		print fileName
		trainXMLFiles.append(fileName + ".xml")

rootDir = "/netscratch/pramanik/OBJECT_DETECTION/detectron_old/lib/datasets/data/Receipts/Annotations_xml"
generateVOC2Json(rootDir, trainXMLFiles)

## VOC xml to YOLOv4 txt

In [ ]:
# 출처: https://bblib.net/entry/convert-voc-to-yolo-xml-to-yolo-xml-to-txt

import glob
import os
import pickle
import xml.etree.ElementTree as ET
from os import listdir, getcwd
from os.path import join

dirs = ['annotation']
classes = ['polyp']


def getImagesInDir(dir_path):
    image_list = []
    for filename in glob.glob(dir_path + '/*.png'):
        image_list.append(filename)
    for filename in glob.glob(dir_path + '/*.jpg'):
        image_list.append(filename)

    return image_list


def convert(size, box):
    dw = 1./(size[0])
    dh = 1./(size[1])
    x = (box[0] + box[1])/2.0 - 1
    y = (box[2] + box[3])/2.0 - 1
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return (x,y,w,h)

def convert_annotation(dir_path, output_path, image_path):
    basename = os.path.basename(image_path)
    basename_no_ext = os.path.splitext(basename)[0]

    in_file = open(dir_path + '/' + basename_no_ext + '.xml' ,encoding='UTF8')
    out_file = open(output_path + basename_no_ext + '.txt', 'w' ,encoding='UTF8')
    tree = ET.parse(in_file)
    root = tree.getroot()
    size = root.find('size')
    w = int(size.find('width').text)
    h = int(size.find('height').text)

    for obj in root.iter('object'):
        difficult = obj.find('difficult').text
        cls = obj.find('name').text
        if cls not in classes or int(difficult)==1:
            continue
        cls_id = classes.index(cls)
        xmlbox = obj.find('bndbox')
        b = (float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text))
        bb = convert((w,h), b)
        out_file.write(str(cls_id) + " " + " ".join([str(a) for a in bb]) + '\n')


In [ ]:

cwd = getcwd()

for dir_path in dirs:
    output_path = path +'/yolo/'
    if not os.path.exists(output_path):
        os.makedirs(output_path)

    image_paths = getImagesInDir(path)
    print(path)
    list_file = open(path + '.txt', 'w')

    for image_path in image_paths:
        list_file.write(image_path + '\n')
        convert_annotation(path, output_path, image_path)
        
    list_file.close()

    print("Finished processing: " + dir_path)



/content/drive/MyDrive/Bbox_0002
Finished processing: annotation


In [ ]:
cnt_txt = []
for dir_path, dir_names, file_names in os.walk(path):
  for file_name in file_names:
    if file_name.endswith('.txt'):
      cnt_txt.append(file_name)

# cnt_img : 195
print(len(cnt_txt))

195


## directory search

In [ ]:
# 출처: https://deepbaksuvision.github.io/Modu_ObjectDetection/posts/02_01_PASCAL_VOC.html

import os
import sys

dataset_path = sys.argv[1]

IMAGE_FOLDER = "JPEGImages"
ANNOTATIONS_FOLDER = "Annotations"

ann_root, ann_dir, ann_files = next(os.walk(os.path.join(dataset_path, ANNOTATIONS_FOLDER)))

print("ROOT : {}\n".format(ann_root))
print("DIR : {}\n".format(ann_dir))
print("FILES : {}\n".format(ann_files))

# 위의 스크립트는 PASCAL VOC 데이터셋 폴더를 기준으로 작성되었습니다.
# 파라미터로 PASCAL VOC Object Detection dataset 경로를 받습니다.
# os.walk 함수를 이용하여 Annotations폴더의 경로, 폴더 리스트, 파일 리스트정보를 담은 iterator를 반환합니다.
# next함수를 이용하여 iterator의 첫번째 값을 가지고 옵니다.
# (os.walk iterator의 첫번째 값은 3개의 값을 리턴하는데, 차례대로 경로, 폴더 리스트, 파일 리스트입니다.)

## xml파일에서 파일 경로,이름 추출

In [ ]:
# xml파일과 이미지파일의 파일 경로+ 파일이름 추출해서 리스트

path3 = '/content/drive/MyDrive/sidewalk_bbox/Bbox_1/'
xml_files = []
img_files = []
dontknow = []
for dir_path, dir_names, file_names in os.walk(path3):
  for file_name in file_names:
    short_path = str(dir_path)[44:]
    if file_name.endswith('.xml'):
      xml_files.append(short_path+'/'+file_name)
    elif file_name.endswith('.jpg'):
      img_files.append(short_path+'/'+file_name)
    elif file_name.endswith('.png'):
      img_files.append(short_path+'/'+file_name)
    else :
      dontknow.append(file_name)
print(xml_files)
print(img_files)
print(dontknow)

In [ ]:
# 라벨 리스트만 추출 (AI hub 홈페이지에 자세한 설명이 있어서 필요한 코드는 아님)

import xml.etree.ElementTree as ET
from xml.etree.ElementTree import Element, dump, ElementTree

def label_tags(xmlname):
  xml_data = open((os.path.join(path3,xmlname)), 'r').read()
  root = ET.XML(xml_data)
  labels = []
  for elem in root.iter("label"):
    # print(elem.find('name').text)
    labels.append(elem.find('name').text)
  return labels

label_list = []
for xml in xml_files:
  label_list.extend(label_tags(xml))

label_list = list(set(label_list))
label_list

['traffic_sign',
 'barricade',
 'stop',
 'carrier',
 'table',
 'parking_meter',
 'tree_trunk',
 'truck',
 'bus',
 'dog',
 'bollard',
 'movable_signage',
 'motorcycle',
 'potted_plant',
 'bench',
 'traffic_light',
 'cat',
 'person',
 'stroller',
 'scooter',
 'pole',
 'kiosk',
 'car',
 'fire_hydrant',
 'wheelchair',
 'bicycle',
 'chair']

## HDF5 포맷

In [ ]:
# 대용량 파일 처리 - hdf5 포맷 사용 (적용방법을 모름..)
# 출처: https://nuxlear.tistory.com/4

import numpy as np
import h5py
 
with h5py.File(filename, 'w') as f:
 
    f.create_dataset('image', (1000, 32, 32, 3), dtype='float32')    # 1000개의 32x32 RGB 이미지를 담는 데이터 공간을 생성한다. 
    f.create_dataset('label', (1000,), dtype='float32')              # 1000개의 float을 담는 데이터 공간을 생성한다. 
    image_set = f['image']    # 실 데이터 공간에 접근할 변수를 할당한다. 
    label_set = f['label']
    labels = label_set[:]    # 이미지 데이터 전체를 반환하지만, 메모리에 올리지는 않는다. 값에 접근할 때 메모리에 올린다. 
    array = label_set[:50]   # 데이터셋을 numpy array 와 비슷한 형태로 불러올 수 있다. 역시 이 명령으로 메모리에 올라가지는 않는다. 
    label_set[0] = 3.0       # dataset에 값을 대입할 수 있다. 
    label_set[10:20] = np.arange(10.)       # 배열을 직접 대입할 수도 있다. 

from keras.utils.io_utils import HDF5Matrix
 
n_epoch = 1000
batch_size = 32
split_pos = 800
 
x_data = HDF5Matrix(filename, 'image')    # 위에서 생성한 HDF5 파일의 image 경로의 데이터를 가져오게 된다. 
x_train = HDF5Matrix(filename, 'image', end=split_pos)    # HDF5 파일의 데이터 중 일부만 가져오는 것도 가능하다. 
x_test = HDF5Matrix(filename, 'image', start=split_pos)
y_train = HDF5Matrix(filename, 'label', end=split_pos)
y_test = HDF5Matrix(filename, 'label', start=split_pos)
 
# 이미 .compile() 이 된 모델이라고 가정하자
model.fit(x_train, y_train, epochs=n_epoch, batch_size=batch_size, validation_data=(x_test, y_test))    # numpy array 를 쓰듯이 사용하면 된다. 
